<a href="https://colab.research.google.com/github/SameekshaNalla/Sameeksha_INFO5731_Fall2024/blob/main/nalla_sameeksha_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [ ]:
#!/usr/bin/env python3
"""
Q1 – Collect all information of the narrators in the Densho Digital Repository
"""

import csv
import json
import time
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError

API_ROOT = "https://ddr.densho.org/api/0.2/narrator/"
OUT_PATH = "/Users/19452/Downloads/densho_narrators.csv"

HEADERS = {
    "Accept": "application/json",
    "User-Agent": "INFO5731-Assignment/1.0"
}

FIELDS = [
    "id", "display_name", "first_name", "last_name",
    "birth_location", "b_date", "d_date", "gender", "generation",
    "ethnicity", "nationality", "bio", "created", "modified", "resource_uri"
]

RETRIABLE = {429, 500, 502, 503, 504}


def fetch_json(url: str, retries: int = 5, base_delay: float = 1.0) -> dict:
    delay = base_delay
    for _ in range(retries):
        try:
            req = Request(url, headers=HEADERS)
            with urlopen(req, timeout=30) as resp:
                return json.loads(resp.read().decode("utf-8"))
        except HTTPError as e:
            if e.code in RETRIABLE:
                time.sleep(delay)
                delay *= 2
                continue
            raise
        except URLError:
            time.sleep(delay)
            delay *= 2
    raise RuntimeError(f"Request failed after {retries} attempts: {url}")


def normalize(rec: dict) -> dict:
    row = {k: "" for k in FIELDS}
    row["id"] = rec.get("id") or rec.get("pk") or ""
    row["display_name"] = (rec.get("display_name") or "").strip()
    row["first_name"] = (rec.get("first_name") or "").strip()
    row["last_name"] = (rec.get("last_name") or "").strip()
    row["birth_location"] = (rec.get("birth_location") or "").strip()
    row["b_date"] = (rec.get("b_date") or rec.get("birth_date") or "").strip()
    row["d_date"] = (rec.get("d_date") or rec.get("death_date") or "").strip()
    row["gender"] = (rec.get("gender") or "").strip()
    row["generation"] = (rec.get("generation") or "").strip()
    row["ethnicity"] = (rec.get("ethnicity") or "").strip()
    row["nationality"] = (rec.get("nationality") or "").strip()

    bio = rec.get("bio") or rec.get("biography") or rec.get("summary") or rec.get("description") or ""
    row["bio"] = bio[:6000]  # keep CSV manageable

    row["created"] = (rec.get("created") or "").strip()
    row["modified"] = (rec.get("modified") or "").strip()

    links = rec.get("links") or {}
    row["resource_uri"] = links.get("html") or links.get("json") or rec.get("resource_uri") or ""
    return row


def page_items_and_next(payload: dict):
    # Support both possible payload styles
    items = payload.get("objects") or payload.get("results") or []
    nxt = payload.get("next_api") or payload.get("next")
    return items, nxt


def collect_all() -> list[dict]:
    url = API_ROOT
    rows: list[dict] = []
    seen = set()
    pages = 0

    while url:
        data = fetch_json(url)
        items, url = page_items_and_next(data)
        for rec in items:
            nid = rec.get("id") or rec.get("pk")
            if nid in seen:
                continue
            seen.add(nid)
            rows.append(normalize(rec))
        pages += 1
        time.sleep(0.4)  # be polite

    print(f"Fetched {len(rows)} narrators across {pages} pages.")
    return rows


def save_csv(rows: list[dict], path: str):
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=FIELDS)
        writer.writeheader()
        writer.writerows(rows)


def main():
    rows = collect_all()
    save_csv(rows, OUT_PATH)
    print(f"Saved: {OUT_PATH}")


if __name__ == "__main__":
    main()

Fetched 1009 narrators across 41 pages.
Saved: /Users/19452/Downloads/densho_narrators.csv


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
#!/usr/bin/env python3

import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# setup
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)
STOP = set(stopwords.words("english"))
STEM = PorterStemmer()
LEMM = WordNetLemmatizer()

try:
    import spacy
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "textcat"])
    USE_SPACY = True
except Exception:
    USE_SPACY = False

IN_PATH  = "/Users/19452/Downloads/densho_narrators.csv"
OUT_PATH = "/Users/19452/Downloads/densho_narrators_clean.csv"

_non_alnum = re.compile(r"[^A-Za-z0-9\s]+")
_numbers   = re.compile(r"\b\d+\b")
_multi_ws  = re.compile(r"\s+")

def preview(df, col, n=3):
    print(f"\n{col}:")
    for i, s in enumerate(df[col].head(n).tolist(), 1):
        print(f"{i}. {s}")

df = pd.read_csv(IN_PATH)
df["bio"] = df["bio"].fillna("").astype(str)

# (1) Remove noise
df["bio_no_noise"] = (
    df["bio"].str.replace(_non_alnum, " ", regex=True)
             .str.replace(_multi_ws, " ", regex=True)
             .str.strip()
)
preview(df, "bio_no_noise")

# (2) Remove numbers
df["bio_no_numbers"] = (
    df["bio_no_noise"].str.replace(_numbers, " ", regex=True)
                      .str.replace(_multi_ws, " ", regex=True)
                      .str.strip()
)
preview(df, "bio_no_numbers")

# (4) Lowercase
df["bio_lower"] = df["bio_no_numbers"].str.lower()
preview(df, "bio_lower")

# (3) Remove stopwords
df["bio_no_stop"] = df["bio_lower"].apply(lambda t: " ".join(w for w in t.split() if w not in STOP))
preview(df, "bio_no_stop")

# (5) Stemming
df["bio_stemmed"] = df["bio_no_stop"].apply(lambda t: " ".join(STEM.stem(w) for w in t.split()))
preview(df, "bio_stemmed")

# (6) Lemmatization
if USE_SPACY:
    df["bio_lemma"] = df["bio_no_stop"].apply(lambda t: " ".join(tok.lemma_ for tok in nlp(t)))
else:
    df["bio_lemma"] = df["bio_no_stop"].apply(lambda t: " ".join(LEMM.lemmatize(w) for w in t.split()))
preview(df, "bio_lemma")

df.to_csv(OUT_PATH, index=False)
print(f"\nSaved: {OUT_PATH}")


bio_no_noise:
1. Nisei female Born May 9 1927 in Selleck Washington Spent much of childhood in Beaverton Oregon where father owned a farm Influenced at an early age by parents conversion to Christianity During World War II removed to the Portland Assembly Center Oregon and the Minidoka concentration camp Idaho After the war worked to establish a successful volunteer program to feed the homeless in Seattle Washington
2. Nisei male Born June 12 1921 in Seattle Washington Grew up in an area of Seattle with few other Japanese Americans and was attending the University of Washington when Japan bombed Pearl Harbor Removed to the Puyallup Assembly Center Washington and the Minidoka concentration camp Idaho Left Minidoka several times on temporary work leave to work on farms in the area Suffered tragic loss in camp when father got lost outside collecting wood and perished in the elements Volunteered for the army and served in the signal corps eventually working for the Civil Aeronautics Admin

# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
#!/usr/bin/env python3

import pandas as pd
import nltk
from collections import Counter
import spacy

# Download required NLTK resources
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)
nltk.download("averaged_perceptron_tagger", quiet=True)
nltk.download("averaged_perceptron_tagger_eng", quiet=True)

# spaCy for parsing + NER
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    raise RuntimeError("Run: python -m spacy download en_core_web_sm")

IN_PATH = "/Users/19452/Downloads/densho_narrators_clean.csv"

df = pd.read_csv(IN_PATH)
texts = df["bio_lemma"].dropna().astype(str).tolist()

# ---- (1) POS Tagging ----
sample = " ".join(texts[:5])
tokens = nltk.word_tokenize(sample)
tagged = nltk.pos_tag(tokens)

pos_map = {
    "NN": "Noun", "NNS": "Noun", "NNP": "Noun", "NNPS": "Noun",
    "VB": "Verb", "VBD": "Verb", "VBG": "Verb", "VBN": "Verb", "VBP": "Verb", "VBZ": "Verb",
    "JJ": "Adj", "JJR": "Adj", "JJS": "Adj",
    "RB": "Adv", "RBR": "Adv", "RBS": "Adv"
}

counts = Counter()
for _, tag in tagged:
    if tag in pos_map:
        counts[pos_map[tag]] += 1

print("\n(1) POS Counts:")
for k in ["Noun", "Verb", "Adj", "Adv"]:
    print(f"{k}: {counts[k]}")

# ---- (2) Parsing ----
doc = nlp(texts[0])

print("\n(2) Dependency Parse:")
for tok in doc:
    print(f"{tok.text:15} {tok.dep_:12} {tok.head.text:15}")

print("\nExample sentence:")
print(list(doc.sents)[0].text)
print("Dependency parsing shows word-to-word relations.")
print("Constituency parsing (not in spaCy) shows phrase hierarchy.")

# ---- (3) Named Entity Recognition ----
all_ents = Counter()
for t in texts[:20]:
    d = nlp(t)
    for ent in d.ents:
        all_ents[ent.label_] += 1

print("\n(3) Named Entities (counts):")
for ent, c in all_ents.most_common():
    print(f"{ent}: {c}")


(1) POS Counts:
Noun: 136
Verb: 37
Adj: 51
Adv: 11

(2) Dependency Parse:
nisei           amod         bear           
female          amod         bear           
bear            nsubj        selleck        
may             aux          selleck        
selleck         ROOT         selleck        
washington      dobj         selleck        
spend           ccomp        selleck        
much            amod         beaverton      
childhood       compound     beaverton      
beaverton       compound     father         
oregon          compound     father         
father          nsubj        own            
own             xcomp        spend          
farm            compound     influence      
influence       dobj         own            
early           amod         age            
age             compound     ii             
parent          compound     ii             
conversion      compound     ii             
christianity    compound     ii             
world           compound 

# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [ ]:
# Q4 – Part 1: Scrape GitHub Marketplace (Actions) to CSV
#!/usr/bin/env python3

import csv
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE   = "https://github.com"
URL    = "https://github.com/marketplace?type=actions&page={}"
OUT    = "/Users/19452/Downloads/github_marketplace_actions.csv"
FIELDS = ["product_name", "description", "url", "page"]
HEADERS = {"User-Agent": "Mozilla/5.0 (INFO5731-Assignment)"}

MAX_ITEMS   = 1000          # target size
REQUEST_GAP = 1.0           # polite delay (seconds)
TIME_LIMIT  = 15 * 60       # overall cap (seconds)

def parse_cards(html):
    soup = BeautifulSoup(html, "html.parser")
    rows = []
    cards = soup.select("article, li, div.marketplace-item, [data-testid='marketplace-listing-card']")
    for c in cards:
        a = c.select_one("a[href*='/marketplace/']") or c.find("a", class_="text-bold") or c.find("a")
        if not a or not a.get("href"):
            continue
        name = a.get_text(strip=True)
        href = urljoin(BASE, a["href"])
        d = c.select_one("p, .color-fg-muted, .marketplace-item-card-desc")
        desc = d.get_text(strip=True) if d else ""
        if name and href:
            rows.append((name, desc, href))
    return rows

def scrape():
    all_rows, start = [], time.time()
    for page in range(1, 10000):
        if time.time() - start > TIME_LIMIT or len(all_rows) >= MAX_ITEMS:
            break
        try:
            r = requests.get(URL.format(page), headers=HEADERS, timeout=30)
            if r.status_code != 200:
                break
            rows = parse_cards(r.text)
            if not rows:
                break
            for name, desc, href in rows:
                all_rows.append({"product_name": name, "description": desc, "url": href, "page": page})
                if len(all_rows) >= MAX_ITEMS:
                    break
        except requests.RequestException:
            time.sleep(2)
            continue
        time.sleep(REQUEST_GAP)
    return all_rows[:MAX_ITEMS]

def main():
    data = scrape()
    with open(OUT, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=FIELDS)
        w.writeheader()
        w.writerows(data)
    print(f"Saved {len(data)} rows → {OUT}")

if __name__ == "__main__":
    main()

Saved 1000 rows → /Users/19452/Downloads/github_marketplace_actions.csv


In [ ]:
#!/usr/bin/env python3

import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# resources
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

IN  = "/Users/19452/Downloads/github_marketplace_actions.csv"
OUT = "/Users/19452/Downloads/github_marketplace_actions_clean.csv"

STOP = set(stopwords.words("english"))
LEMM = WordNetLemmatizer()

def clean_text(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = re.sub(r"<[^>]+>", " ", t)
    t = re.sub(r"[^A-Za-z\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip().lower()
    toks = [w for w in nltk.word_tokenize(t) if w not in STOP]
    toks = [LEMM.lemmatize(w) for w in toks]
    return " ".join(toks)

def main():
    df = pd.read_csv(IN)

    # preprocess
    df["description_clean"] = df["description"].apply(clean_text)

    # data quality: report
    print("\nMissing values per column:\n", df.isna().sum())
    print("\nDuplicate rows:", df.duplicated().sum())

    # data quality: fix
    df = df.drop_duplicates()
    df["product_name"] = df["product_name"].fillna("")
    df["description"]   = df["description"].fillna("")
    df["url"]           = df["url"].fillna("")
    df["page"]          = df["page"].fillna(0).astype(int)

    df.to_csv(OUT, index=False)
    print(f"\nSaved → {OUT}")

if __name__ == "__main__":
    main()


Missing values per column:
 product_name            0
description          1000
url                     0
page                    0
description_clean       0
dtype: int64

Duplicate rows: 77

Saved → /Users/19452/Downloads/github_marketplace_actions_clean.csv


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [ ]:
#Q5 – PART 1: Collect tweets (hashtags: ML/AI) → CSV

#!/usr/bin/env python3

import time, csv, tweepy

# paste your keys/tokens
API_KEY = "nUA4bSM2ZIGKL7uTQapXmK6Rw"
API_SECRET = "SQ9T7Ytz7xrOzVBClRYU7edb0BIzuVZF0PwrNcUpPriywSd4qm"
BEARER = "AAAAAAAAAAAAAAAAAAAAALm%2F4QEAAAAAESJZ7CuPOIGsqsXEC5wqnxFQw7c%3DETiTZek7V0vrH5iDmBi37PXR5ADWcxfNQ8XAVtoCDP7yS3Ulht"

OUT = "/Users/19452/Downloads/twitter_ml_ai_raw.csv"
FIELDS = ["tweet_id", "username", "text", "created_at"]

QUERY = "(#machinelearning OR #artificialintelligence OR #ml OR #ai) lang:en -is:retweet"
MAX_TWEETS = 90         # stay under your 100 reads/month cap
PAGE_DELAY = 1.0

def fetch_rows(client, limit):
    rows, seen = [], set()
    pages = tweepy.Paginator(
        client.search_recent_tweets,
        query=QUERY,
        tweet_fields=["id","text","created_at","author_id","lang"],
        user_fields=["username"],
        expansions=["author_id"],
        max_results=100,
    )
    for page in pages:
        if not page.data: break
        users = {u.id: u.username for u in (page.includes.get("users") or [])}
        for t in page.data:
            if t.lang != "en": continue
            if t.id in seen:    continue
            seen.add(t.id)
            rows.append({
                "tweet_id": str(t.id),
                "username": users.get(t.author_id, ""),
                "text": (t.text or "").replace("\r"," ").replace("\n"," ").strip(),
                "created_at": t.created_at.isoformat() if t.created_at else ""
            })
            if len(rows) >= limit: return rows
        time.sleep(PAGE_DELAY)
    return rows

def main():
    if not BEARER: raise RuntimeError("Add your bearer token.")
    client = tweepy.Client(
        bearer_token=BEARER,
        consumer_key=API_KEY,
        consumer_secret=API_SECRET,
        wait_on_rate_limit=True
    )
    rows = fetch_rows(client, MAX_TWEETS)
    with open(OUT, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=FIELDS)
        w.writeheader(); w.writerows(rows)
    print(f"Saved {len(rows)} tweets → {OUT}")

if __name__ == "__main__":
    main()

Saved 90 tweets → /Users/19452/Downloads/twitter_ml_ai_raw.csv


In [ ]:
#!/usr/bin/env python3

import re, pandas as pd, nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

IN  = "/Users/19452/Downloads/twitter_ml_ai_raw.csv"
OUT = "/Users/19452/Downloads/twitter_ml_ai_clean.csv"

STOP = set(stopwords.words("english"))
LEMM = WordNetLemmatizer()

URL_RE   = re.compile(r"https?://\S+")
MENT_RE  = re.compile(r"@[A-Za-z0-9_]+")
HASH_RE  = re.compile(r"#")
EMOJI_RE = re.compile(r"[\U00010000-\U0010ffff]")
NON_ALPH = re.compile(r"[^a-z\s]")
MULTI_WS = re.compile(r"\s+")

def clean_text(t: str) -> str:
    if not isinstance(t, str): return ""
    t = URL_RE.sub(" ", t)
    t = MENT_RE.sub(" ", t)
    t = HASH_RE.sub(" ", t)
    t = EMOJI_RE.sub(" ", t)
    t = t.lower()
    t = NON_ALPH.sub(" ", t)
    t = MULTI_WS.sub(" ", t).strip()
    toks = [w for w in nltk.word_tokenize(t) if w not in STOP]
    toks = [LEMM.lemmatize(w) for w in toks]
    return " ".join(toks)

def main():
    df = pd.read_csv(IN, dtype={"tweet_id": str}).fillna({"username":"", "text":"", "created_at":""})
    df["text_clean"] = df["text"].apply(clean_text)

    print("\nMissing values per column:\n", df.isna().sum())
    print("\nDuplicate tweet_ids:", df.duplicated(subset=["tweet_id"]).sum())

    df = df.drop_duplicates(subset=["tweet_id"])
    df.to_csv(OUT, index=False)
    print(f"\nSaved → {OUT}")

if __name__ == "__main__":
    main()


Missing values per column:
 tweet_id      0
username      0
text          0
created_at    0
text_clean    0
dtype: int64

Duplicate tweet_ids: 0

Saved → /Users/19452/Downloads/twitter_ml_ai_clean.csv


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog